# * VINSIGHT : Daily Status

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## LOAD_DATE Summary

In [2]:
# Input parameter
v_start_date = 20250201
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ PPN_TM, AREA_TYPE
        , COUNT(DISTINCT METRIC_CD) CNT_METRIC, COUNT(1) ROW_CNT
        , MIN(TM_KEY_DAY) MIN_DT, MAX(TM_KEY_DAY) MAX_DT, MAX(ACTUAL_AS_OF) LAST_ACTUAL
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE CENTER IN ('Revenue', 'Sales', 'Profitability')
    AND TM_KEY_DAY >= {v_start_date} 
    GROUP BY PPN_TM, AREA_TYPE
    ORDER BY 1,2
""")


try:
    # Get : LOAD_DATE Summary
    src_cur.execute(query)
    rows = src_cur.fetchall()
    print(f'\nCurrent LOAD_DATE Summary...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    tmp_src_df = chk_src_df.copy()
    print(f'\n{tmp_src_df.to_string(line_width=100)}')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> v_start_date: 20250201

Current LOAD_DATE Summary...

                PPN_TM AREA_TYPE  CNT_METRIC  ROW_CNT    MIN_DT    MAX_DT  LAST_ACTUAL
0  2025-03-04 23:53:32      CCAA         166  4775488  20250201  20250303     20250302
1  2025-03-05 08:39:02         C         530    16960  20250201  20250304     20250303
2  2025-03-05 08:39:02         G         317    81152  20250201  20250304     20250303
3  2025-03-05 08:39:02         P         341    10912  20250201  20250304     20250303
4  2025-03-05 08:39:02         Z          29     6496  20250201  20250304     20250302
5  2025-03-05 08:41:07         C           4      128  20250201  20250304     20250304
6  2025-03-05 08:41:07         G           4     1024  20250201  20250304     20250304
7  2025-03-05 08:41:07         P           4      128  20250201  20250304     20250304
8  2025-03-05 10:46:31         H         316   667392  20250201  20250304     20250303
9  2025-03-05 10:50:15         H           4     